# But prise en main des objets créés par mes collègues et lancement d'un entrapînement à partir de la BDTOPO ou du RIL en tant que LAbel ??

In [137]:
%load_ext autoreload
%autoreload 2

In [145]:
import sys
sys.path.append('../src')
from satellite_image import SatelliteImage
from utils import *
from utils import get_root_path


In [161]:
import yaml
import re
import s3fs
import numpy as np

In [151]:
os.environ["AWS_ACCESS_KEY_ID"]=
os.environ["AWS_SECRET_ACCESS_KEY"]=
os.environ["AWS_S3_ENDPOINT"]='minio.lab.sspcloud.fr'
os.environ["AWS_SESSION_TOKEN"]=

In [153]:
fs = get_file_system() # bien définir les variables d'environnement en amont !

In [154]:
fs.ls('projet-slums-detection/Donnees/RIL/')

In [195]:
environment = get_environment()

root_path = get_root_path()
bucket = environment["bucket"]
path_s3_cayenne_data = environment["sources"]["PLEIADES"]
path_local_cayenne_data = os.path.join(root_path, environment["local-path"]["PLEIADES"])
bucket = environment["bucket"]

path_s3_pleiades_data = environment["sources"]["PLEIADES"]
path_s3_bdtopo_data = environment["sources"]["BDTOPO"][2022]["guyane"]
path_local_pleiades_data = environment["local-path"]["PLEIADES"]
path_local_bdtopo_data = environment["local-path"]["BDTOPO"][2022]["guyane"]


In [ ]:

# DL PLEIADE
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data}",
        lpath=f"../{path_local_pleiades_data}",
        recursive=True)


In [196]:
# DL BDTOPO
fs.download(
        rpath=f"{bucket}/{path_s3_bdtopo_data}",
        lpath=f"../{path_local_bdtopo_data}",
        recursive=True)


## Chargement données pleiades

In [35]:
filename = 'Cayenne_200722/16bits/ORT_2022072050325085_U22N/ORT_2022072050325085_0353_0545_U22N_16Bits.jp2'
date = datetime.strptime(re.search(r'ORT_(\d{8})', filename).group(1), '%Y%m%d')
date

In [80]:
image = SatelliteImage.from_raster(
        filename,
        date = date, 
        n_bands = 4,
        dep = "973"
    )

In [58]:
list_images = image.split(250)
list_images[0].array
len(list_images)

In [61]:
image.plot([0,1,2])
image.plot([3,1,2])

## Représenter la liste des images sous forme de grille (ajouter une fonction)

In [71]:
[im.bounds for im in list_images[1:10]] # Il suffit d'ordonner ça !

In [76]:
image.bounds[0]
image.bounds[1]
image.bounds[2]
# Plus qu'à utiliser la fonction que j'ai créée pour l'affichage d'une grille ?
# mettre certains notebooks dans du #old !

In [78]:
64*256 # nb de patchs utilisables pour entrainer un modèle, why not ?

In [86]:
#image.plot([0,1,2])
image.normalized # ok le plot normalise l'image donc attention  ? j'aurais tendanc e à l'enlever moi

## Récupération des labellisations

In [178]:
from labeler import RILLabeler

Définition d'une instance de labeller (on donneune datez et un département et on va chercher les labels associés RIL à côté)

In [164]:
labeler = RILLabeler(date, dep = "973", buffer_size= 20)

In [165]:
mask = labeler.create_segmentation_label(image) # va chercher les données et rasterise les bounding box

In [201]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(np.transpose(image.array, (1, 2, 0))[:,:,:3])
ax.imshow(mask, alpha=0.3) # magnifique ! Faire une fonction  de représentation  ? # faire du découpage ? ou une fonction de sélection aléatoire d'une tuile ?

# Plus qu'à faire la même chose sur le mont baduel :)

In [175]:
filename = 'Cayenne_200722/16bits/ORT_2022072050325085_U22N/ORT_2022072050325085_0354_0545_U22N_16Bits.jp2'
date = datetime.strptime(re.search(r'ORT_(\d{8})', filename).group(1), '%Y%m%d')
date

image = SatelliteImage.from_raster(
        filename,
        date = date, 
        n_bands = 4,
        dep = "973"
    )

image.plot([3,1,2])

## Ril

In [202]:

labeler = RILLabeler(date, dep = "973", buffer_size= 20)
mask = labeler.create_segmentation_label(image)

fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(np.transpose(image.array, (1, 2, 0))[:,:,:3])
ax.imshow(mask, alpha=0.3) # magnifique ! Faire une fonction  de représentation  ? # faire du découpage ? ou une fonction de sélection aléatoire d'une tuile ?

## BD TOPO

In [179]:
from labeler import BDTOPOLabeler

In [203]:
labeler = BDTOPOLabeler(date, dep = "973") # charghement lenbt nécessite d'avoir downloader la BDTOPO avant. fluidifier ça ?
mask = labeler.create_segmentation_label(image)

fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(np.transpose(image.array, (1, 2, 0))[:,:,:3])
ax.imshow(mask, alpha=0.3) # magnifique ! Faire une fonction  de représentation  ? # faire du découpage ? ou une fonction de sélection aléatoire d'une tuile ?

Le moint Baduel a bien été cartographié !

## Mélange images Labels !

In [180]:
from labeled_satellite_image import SegmentationLabeledSatelliteImage

## TO DO 
- fonctions de représentations ?
- création d'une classe Dataset particulière avec les aleas 
- ajout de fonctions de représentations
- bérifier qu'on retrouve bien la géométrie à partir de l'image et vice versa
- Quand on fait du plot, ne pas normaliser l'arraymais en faire une copie pour normaliser ensuite et représenter la normalisation sera différente pour les réseaux
- Faire une fonction create bounding box pour yolo etc.. c'est l'objet de create detection label , faire une fonction de représentation ?
- access Vault ?
- Faire un notebook de prise en main pour les stagiaires
- chargement super lent de la BDTOPO, que faire ?